In [8]:
import pandas as pd

pd.set_option('display.float_format', lambda x: f'{x:.2f}')

df = pd.read_csv("data/DataIndonesia.csv")

df


,DATASET,SERIES_CODE,OBS_MEASURE,COUNTRY,INDICATOR,FREQUENCY,SCALE,2000,2001,2002,...,2016,2017,2018,2019,2020,2021,2022,2023,2024,2025
0,IMF.APD:APDREO(6.0.0),IDN.GGXCNL_GDP.A,OBS_VALUE,Indonesia,"Net lending (+) / net borrowing (-), General g...",Annual,Units,-1.87,-1.76,-0.58,...,-2.56,-2.26,-1.69,-2.09,-6.07,-4.40,-2.31,-1.64,NaN,NaN
1,IMF.APD:APDREO(6.0.0),IDN.NGDP_R_PPP_PC_PCH.A,OBS_VALUE,Indonesia,"Gross domestic product (GDP), Constant prices,...",Annual,Units,3.78,2.19,3.03,...,3.69,3.92,4.06,3.94,-3.26,2.76,4.15,3.93,5.03,NaN
2,IMF.APD:APDREO(6.0.0),IDN.BCA_GDP_BP6.A,OBS_VALUE,Indonesia,"Current account balance (credit less debit), P...",Annual,Units,4.45,3.95,3.68,...,-1.82,-1.59,-2.94,-2.71,-0.42,0.30,1.00,-0.15,NaN,NaN
3,IMF.APD:APDREO(6.0.0),IDN.PCPIE_PCH.A,OBS_VALUE,Indonesia,"All Items, Consumer price index (CPI), End-of-...",Annual,Units,9.33,12.58,9.94,...,3.03,3.61,3.16,2.58,1.68,1.88,5.41,2.81,1.57,2.72
4,IMF.APD:APDREO(6.0.0),IDN.NGDP_RPCH.A,OBS_VALUE,Indonesia,"Gross domestic product (GDP), Constant prices,...",Annual,Units,4.98,3.64,4.50,...,5.03,5.07,5.17,5.02,-2.07,3.70,5.31,5.05,NaN,NaN
5,IMF.APD:APDREO(6.0.0),IDN.LUR.A,OBS_VALUE,Indonesia,Unemployment rate,Annual,Units,6.08,8.10,9.06,...,5.61,5.50,5.34,5.28,7.07,6.49,5.86,5.32,4.91,4.85
6,IMF.APD:APDREO(6.0.0),IDN.PCPI_PCH.A,OBS_VALUE,Indonesia,"All Items, Consumer price index (CPI), Period ...",Annual,Units,3.69,11.47,11.91,...,3.53,3.81,3.29,2.82,2.03,1.56,4.14,3.71,1.57,2.72


In [9]:
df = df.drop(columns=["DATASET", "SERIES_CODE", "OBS_MEASURE"])


In [10]:
df

,COUNTRY,INDICATOR,FREQUENCY,SCALE,2000,2001,2002,2003,2004,2005,...,2016,2017,2018,2019,2020,2021,2022,2023,2024,2025
0,Indonesia,"Net lending (+) / net borrowing (-), General g...",Annual,Units,-1.87,-1.76,-0.58,-1.08,-0.26,0.46,...,-2.56,-2.26,-1.69,-2.09,-6.07,-4.40,-2.31,-1.64,NaN,NaN
1,Indonesia,"Gross domestic product (GDP), Constant prices,...",Annual,Units,3.78,2.19,3.03,3.31,3.55,4.21,...,3.69,3.92,4.06,3.94,-3.26,2.76,4.15,3.93,5.03,NaN
2,Indonesia,"Current account balance (credit less debit), P...",Annual,Units,4.45,3.95,3.68,3.17,1.88,0.51,...,-1.82,-1.59,-2.94,-2.71,-0.42,0.30,1.00,-0.15,NaN,NaN
3,Indonesia,"All Items, Consumer price index (CPI), End-of-...",Annual,Units,9.33,12.58,9.94,5.19,6.38,17.14,...,3.03,3.61,3.16,2.58,1.68,1.88,5.41,2.81,1.57,2.72
4,Indonesia,"Gross domestic product (GDP), Constant prices,...",Annual,Units,4.98,3.64,4.50,4.78,5.03,5.69,...,5.03,5.07,5.17,5.02,-2.07,3.70,5.31,5.05,NaN,NaN
5,Indonesia,Unemployment rate,Annual,Units,6.08,8.10,9.06,9.67,9.86,11.24,...,5.61,5.50,5.34,5.28,7.07,6.49,5.86,5.32,4.91,4.85
6,Indonesia,"All Items, Consumer price index (CPI), Period ...",Annual,Units,3.69,11.47,11.91,6.79,6.06,10.46,...,3.53,3.81,3.29,2.82,2.03,1.56,4.14,3.71,1.57,2.72


# Menggunakan Model ARIMA


In [11]:
import pandas as pd
from statsmodels.tsa.arima.model import ARIMA
import warnings
warnings.filterwarnings("ignore")

# Load data
df = df.copy()

# 3. Tahun yang dipakai untuk forecasting
years_2000_2023 = [str(y) for y in range(2000, 2024)]  # training untuk 2024
years_2000_2024 = [str(y) for y in range(2000, 2025)]  # training untuk 2025

def arima_forecast(series, steps):
    series = series.dropna()

    if len(series) < 5:
        return [None] * steps

    try:
        model = ARIMA(series, order=(1,1,1))
        res = model.fit()
        preds = res.forecast(steps=steps)
        return preds.tolist()
    except:
        return [None] * steps

# --- Tahap 1: Forecast / isi tahun 2024 --- #
for i, row in df.iterrows():
    if pd.isna(row["2024"]):
        series_2000_2023 = row[years_2000_2023].astype(float)
        pred2024 = arima_forecast(series_2000_2023, steps=1)[0]
        df.at[i, "2024"] = pred2024

# --- Tahap 2: Forecast / isi tahun 2025 --- #
for i, row in df.iterrows():
    if pd.isna(row["2025"]):
        series_2000_2024 = row[years_2000_2024].astype(float)
        pred2025 = arima_forecast(series_2000_2024, steps=1)[0]
        df.at[i, "2025"] = pred2025

df




,COUNTRY,INDICATOR,FREQUENCY,SCALE,2000,2001,2002,2003,2004,2005,...,2016,2017,2018,2019,2020,2021,2022,2023,2024,2025
0,Indonesia,"Net lending (+) / net borrowing (-), General g...",Annual,Units,-1.87,-1.76,-0.58,-1.08,-0.26,0.46,...,-2.56,-2.26,-1.69,-2.09,-6.07,-4.40,-2.31,-1.64,-1.97,-2.14
1,Indonesia,"Gross domestic product (GDP), Constant prices,...",Annual,Units,3.78,2.19,3.03,3.31,3.55,4.21,...,3.69,3.92,4.06,3.94,-3.26,2.76,4.15,3.93,5.03,3.92
2,Indonesia,"Current account balance (credit less debit), P...",Annual,Units,4.45,3.95,3.68,3.17,1.88,0.51,...,-1.82,-1.59,-2.94,-2.71,-0.42,0.30,1.00,-0.15,-0.09,-0.14
3,Indonesia,"All Items, Consumer price index (CPI), End-of-...",Annual,Units,9.33,12.58,9.94,5.19,6.38,17.14,...,3.03,3.61,3.16,2.58,1.68,1.88,5.41,2.81,1.57,2.72
4,Indonesia,"Gross domestic product (GDP), Constant prices,...",Annual,Units,4.98,3.64,4.50,4.78,5.03,5.69,...,5.03,5.07,5.17,5.02,-2.07,3.70,5.31,5.05,4.85,4.85
5,Indonesia,Unemployment rate,Annual,Units,6.08,8.10,9.06,9.67,9.86,11.24,...,5.61,5.50,5.34,5.28,7.07,6.49,5.86,5.32,4.91,4.85
6,Indonesia,"All Items, Consumer price index (CPI), Period ...",Annual,Units,3.69,11.47,11.91,6.79,6.06,10.46,...,3.53,3.81,3.29,2.82,2.03,1.56,4.14,3.71,1.57,2.72


# Menggunakan Model ETS untuk TAHUN 2025-2026

In [12]:
from statsmodels.tsa.holtwinters import ExponentialSmoothing
import pandas as pd
import numpy as np

# -------------------------------------------------------------------
# 1. Definisi tahun historis & tahun forecast
#    (setelah blok ARIMA kamu dijalankan)
# -------------------------------------------------------------------
years_2000_2025 = [str(y) for y in range(2000, 2026)]  # 2000–2025
forecast_years   = [str(y) for y in range(2026, 2031)]  # 2026–2030

# Tambahkan kolom kosong untuk forecast kalau belum ada
for year in forecast_years:
    if year not in df.columns:
        df[year] = None

# -------------------------------------------------------------------
# 2. ETS Forecast per baris (COUNTRY–INDICATOR)
#    Catatan: angka di df (2000–2025) sudah 9.33 = 9.33%
# -------------------------------------------------------------------
for i, row in df.iterrows():
    # Ambil deret historis 2000–2025 (sudah termasuk hasil ARIMA 2024–2025)
    series = row[years_2000_2025].astype(float).dropna()

    # Minimal 5 data supaya model stabil
    if len(series) < 5:
        continue

    try:
        # Model ETS tanpa musiman (data tahunan)
        model = ExponentialSmoothing(series, trend="add", seasonal=None)
        model_fit = model.fit()

        # Forecast 5 tahun (2026–2030)
        preds = model_fit.forecast(len(forecast_years))

        # Masukkan hasil forecast ke df (masih dalam skala yang sama)
        for j, year in enumerate(forecast_years):
            df.at[i, year] = preds.iloc[j]

    except Exception as e:
        print(f"ETS error untuk indikator: {row.get('INDICATOR', '')}")
        print(e)

# -------------------------------------------------------------------
# 3. Ubah dari wide → long
# -------------------------------------------------------------------
all_years = years_2000_2025 + forecast_years

df_long = df.melt(
    id_vars=["COUNTRY", "INDICATOR", "FREQUENCY", "SCALE"],
    value_vars=all_years,
    var_name="TAHUN",
    value_name="VALUE"
)

# Rapikan urutan
df_long = df_long.sort_values(
    ["COUNTRY", "INDICATOR", "TAHUN"]
).reset_index(drop=True)

# Pastikan tipe
df_long["TAHUN"] = df_long["TAHUN"].astype(int)
df_long["VALUE"] = pd.to_numeric(df_long["VALUE"], errors="coerce")

# -------------------------------------------------------------------
# 4. Tambahkan kolom TYPE (Actual / Forecast)
# -------------------------------------------------------------------
# Default: semua dianggap Forecast
df_long["TYPE"] = "Forecast"

# 2000–2023 = Actual untuk semua indikator
mask_hist = df_long["TAHUN"] <= 2023
df_long.loc[mask_hist, "TYPE"] = "Actual"

# Indikator yang punya data aktual di 2024
ind_2024_actual = [
    "Gross domestic product (GDP), Constant prices, Purchasing power parity (PPP) per capita percent change",
    "All Items, Consumer price index (CPI), End-of-period (EoP), percent change",
    "Unemployment rate",
    "All Items, Consumer price index (CPI), Period average, percent change",
]

mask_2024_actual = (df_long["TAHUN"] == 2024) & df_long["INDICATOR"].isin(ind_2024_actual)
df_long.loc[mask_2024_actual, "TYPE"] = "Actual"

# Indikator yang punya data aktual di 2025
ind_2025_actual = [
    "All Items, Consumer price index (CPI), End-of-period (EoP), percent change",
    "Unemployment rate",
    "All Items, Consumer price index (CPI), Period average, percent change",
]

mask_2025_actual = (df_long["TAHUN"] == 2025) & df_long["INDICATOR"].isin(ind_2025_actual)
df_long.loc[mask_2025_actual, "TYPE"] = "Actual"

# 2026–2030 dibiarkan Forecast (pure ETS)

# -------------------------------------------------------------------
# 5. SCALE untuk Power BI
#    Kalau angka sudah 9.33 dan kamu mau tampil 9.33% di Power BI:
#    cukup set format = Percentage di Power BI (tanpa bagi 100)
# -------------------------------------------------------------------
# df_long["VALUE"] = df_long["VALUE"] / 100.0   # pakai ini kalau mau skala 0.0933

# -------------------------------------------------------------------
# 6. Simpan ke CSV (siap di-load ke Power BI)
# -------------------------------------------------------------------
output_path = "data/DataIndonesiaForecast.csv"
df_long.to_csv(output_path, index=False, float_format="%.2f")

df_long


,COUNTRY,INDICATOR,FREQUENCY,SCALE,TAHUN,VALUE,TYPE
0,Indonesia,"All Items, Consumer price index (CPI), End-of-...",Annual,Units,2000,9.33,Actual
1,Indonesia,"All Items, Consumer price index (CPI), End-of-...",Annual,Units,2001,12.58,Actual
2,Indonesia,"All Items, Consumer price index (CPI), End-of-...",Annual,Units,2002,9.94,Actual
3,Indonesia,"All Items, Consumer price index (CPI), End-of-...",Annual,Units,2003,5.19,Actual
4,Indonesia,"All Items, Consumer price index (CPI), End-of-...",Annual,Units,2004,6.38,Actual
...,...,...,...,...,...,...,...
212,Indonesia,Unemployment rate,Annual,Units,2026,4.80,Forecast
213,Indonesia,Unemployment rate,Annual,Units,2027,4.75,Forecast
214,Indonesia,Unemployment rate,Annual,Units,2028,4.70,Forecast
215,Indonesia,Unemployment rate,Annual,Units,2029,4.65,Forecast
